Author: Joshua, Will, Ethan <br />
Summary: Makes a coverage matrix {0,1} if an ambulance station covers a region

#big patch! -> you need to adjust coverage by the regression.

In [71]:
import csv
import collections
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt

In [72]:
REGRESSION_FLAG = True #do you want to include a regression?

In [102]:
# This is the grid object, which is used throughout all data preprocessing.
# It represents the city of Austin through a series of grids.
# It thus makes a tractable way to compute distance between grids, ect. 
class Grid():
    def __init__(self, grid_json):
        self.grid = grid_json
        self.min_lat = self.grid["latitude_min"]
        self.min_lon = self.grid["longitude_min"]
        self.max_lat = self.grid["latitude_max"]
        self.max_lon = self.grid["longitude_max"]
        self.latitude_delta = self.grid["latitude_step"]
        self.longitude_delta = self.grid["longitude_step"]
        self.nrows = math.ceil((self.max_lat - self.min_lat) / self.latitude_delta)
        self.ncols = math.ceil((self.max_lon - self.min_lon) / self.longitude_delta)
        self.times = self.grid["time_matrix"]
        self.census_tract_region_map = self.grid["census_tract_region_mapping"]
        self.region_to_tract = collections.defaultdict(list)
        for census_tract in self.census_tract_region_map:
            for region in self.census_tract_region_map[census_tract]:
                self.region_to_tract[region].append(census_tract)
    def map_point_to_region(self, latitude, longitude):
        return math.floor((latitude-self.min_lat)/self.latitude_delta) * self.ncols  + math.floor((longitude-self.min_lon)/self.longitude_delta)
    def get_representative(self, region_num):
        row_num = region_num//self.ncols
        col_num = region_num - row_num*self.ncols
        lat = self.min_lat + row_num * self.latitude_delta + 0.5*self.latitude_delta
        lon = self.min_lon + col_num * self.longitude_delta + 0.5*self.longitude_delta
        return [lon, lat]
    def get_time(self, region1, region2):
        try:
            return self.times[region1][region2]
        except IndexError:
            return -1
    def region_to_census_tract(self, region):
        try:
            return self.region_to_tract[region]
        except KeyError:
            return "0_0"

In [103]:
# Using smaller distance matrix for hopefully faster runtime of Julia code
with open("../Input_Data/grid_info_smaller.json", "r") as f:
    grid_json = json.load(f)
    
g = Grid(grid_json)


In [104]:
stations = pd.read_csv("../Output_Data/austin_data/stations.csv")

#stations = stations[stations["FACILITY_TYPE"].str.contains("Medic Station")]

numstations = stations["LATITUDE"].size # Should be 51
numregions = g.nrows * g.ncols

coverage_times = np.zeros(shape=(numstations, numregions))
coverage = np.zeros(shape=(numstations, numregions))
print(numregions)
numstations

210


44

In [80]:
region_numbers = range(1, numregions + 1)

station_numbers = ["".join(["x", str(i)]) for i in range(1, numstations + 1)]

coverage = pd.DataFrame(data=coverage, index=station_numbers, columns=region_numbers)
coverage_times = pd.DataFrame(data=coverage_times, index=station_numbers, columns=region_numbers)

In [99]:
max_travel_time = 10 #grid time get time is in seconds. So the threshhold is 15 minutes times 60 seconds

none_count = 0
zero_count = 0
minus_count = 0
shortfall_count = 0
#make 0 if travel time is greater than shortfall time
#travel time is none
#travel time = 0 and not at yourself location
print(str(REGRESSION_FLAG))
for station in range(0, numstations):
    region_of_station = g.map_point_to_region(stations["LATITUDE"][station], stations["LONGITUDE"][station])
    #print(region_of_station)
    for region in range(0, numregions):
        travel_time = g.get_time(region_of_station, region) 
        
        #REGRESSION LINE
        if (travel_time is not None ):
            travel_time = travel_time / 60
            
        if (travel_time is not None ) and (travel_time > 0) and (REGRESSION_FLAG):
            slope = 0.23868463365149437 
            intercept = 1.261777659720721
            ax = np.log(travel_time)*slope
            travel_time = np.exp(ax + intercept)      
           
            
        # travel_times[station][region] = travel_time
        # covered if <= 600 seconds = 10 minutes
        # not covered if >= 600 seconds, not reachable (travel_time is None), 
        #    or invalid region (travel_time == 0 and region_of_station != region)
        #if (travel_time is None) or ( travel_time > max_travel_time ) or (travel_time == 0 and region_of_station != region):
        if(travel_time is None):
            coverage.at["x"+str(station+1), region+1] = 0
            none_count = none_count + 1
        elif (travel_time == 0 ): #== 0 and region_of_station != region
            coverage.at["x"+str(station+1), region+1] = 0
            zero_count = zero_count + 1
        elif (travel_time < 0 ): #== 0 and region_of_station != region
            coverage.at["x"+str(station+1), region+1] = 0
            minus_count = minus_count + 1
        elif (travel_time > max_travel_time) :
            shortfall_count = shortfall_count + 1
            coverage.at["x"+str(station+1), region+1] = 0
        else:
            coverage.at["x"+str(station+1), region+1] = 1


True


44

In [101]:
coverage_np = coverage.to_numpy()
numstations = coverage_np.shape[0]
numregions = coverage_np.shape[1]
print(none_count /  (numstations * numregions) )
print(minus_count /  (numstations * numregions) )
print(zero_count /  (numstations * numregions) )
print(shortfall_count/  (numstations * numregions) )
print(sum(sum(coverage_np)) /  (numstations * numregions))

0.0
0.010207237859573151
0.3891122796164553
0.011950622838343222
0.5887298596856283


#### 
(total count = 142252) <br>
Original -> regression = 2074.0 -> 83748 <br>
taking out -1 travel times: 85200.0 -> 83748 <br>

## Make all regions with no coverage fully covered (why?)
##### Is this related to the 0 min grid travel times? I need to filter out calls outside of the grid region.

In [68]:
g.get_time(1,85)

0

In [95]:
for col in coverage:
    #print(col, sum(coverage[col]))
    if sum(coverage[col]) == 0:
        coverage[col] += 1
        print("\t->", sum(coverage[col]))

	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	

	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	-> 44.0
	

In [97]:
sum(sum(coverage_np)) /  (numstations * numregions)

0.9880493771616567

In [31]:
coverage.to_csv("../Output_Data/austin_data_3200/coverage_regression.csv")
coverage.transpose()
print(numregions)
coverage.shape
# travel_times.to_csv("coverage_times.csv")

3233


(44, 3233)

In [32]:
#sanity check
#this is checking that a station covers at least one region. Which is somewhat obvious
for index, row in coverage.iterrows():
    if sum(row) == 0:
        print("NO COVERAGE: (region#: ", index, "), ", "(census tract: ", g.region_to_census_tract(index), "), ", g.get_representative(index))
    

In [17]:
#coverage.to_csv("Austin_data/coverage_real.csv", header=None, index=False)